<a href="https://colab.research.google.com/github/thor4/crypto/blob/main/get_events.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


Load libraries. Define API key and addresses to scan for emitted events.

In [8]:
import requests
import json
import hashlib

API_KEY = "<replace-me>"

CURVE_VOTING_OWNERSHIP_ADDRESS = "0xE478de485ad2fe566d49342Cbd03E49ed7DB3356"
CURVE_VOTING_PARAMETER_ADDRESS = "0xBCfF8B0b9419b9A88c44546519b1e909cF330399"
CURVE_VOTING_EMERGENCY_ADDRESS = "0x1115c9b3168563354137cDc60efb66552dd50678"
CONTRACT_ADDRESSES = [CURVE_VOTING_OWNERSHIP_ADDRESS, CURVE_VOTING_PARAMETER_ADDRESS, CURVE_VOTING_EMERGENCY_ADDRESS]

ETHERSCAN_API_URL = "https://api.etherscan.io/api"

In [12]:
def get_contract_abi(contract_address):
    payload = {
        "module": "contract",
        "action": "getabi",
        "address": contract_address,
        "apikey": API_KEY
    }
    response = requests.get(ETHERSCAN_API_URL, params=payload)
    data = json.loads(response.text)
    return json.loads(data["result"])

def get_events_emitted_by_contract(contract_address):
    payload = {
        "module": "logs",
        "action": "getLogs",
        "fromBlock": "0",
        "toBlock": "latest",
        "address": contract_address,
        "apikey": API_KEY
    }
    response = requests.get(ETHERSCAN_API_URL, params=payload)
    data = json.loads(response.text)
    result = data["result"]
    if isinstance(result, str):
        return []
    return [log["topics"][0] for log in result]

def build_signature_to_name_mapping(contract_abi): 
    signature_to_name = {} 
    for item in contract_abi: 
        if item["type"] == "event": 
            signature = item["inputs"] 
            signature_str = ",".join([f"{sig['type']} {sig['name']}" for sig in signature]) 
            event_signature = f"{item['name']}({signature_str})" 
            signature_hash = "0x" + hashlib.sha3_256(event_signature.encode()).hexdigest() 
            signature_to_name[signature_hash] = item["name"] 
    return signature_to_name

def main(): 
    event_hashes_dict = {} 
    for contract_address in CONTRACT_ADDRESSES: 
        abi = get_contract_abi(contract_address) 
        signature_to_name = build_signature_to_name_mapping(abi) 
        events = get_events_emitted_by_contract(contract_address) 
        for event_hash in events: 
            event_name = signature_to_name.get(event_hash, "Unknown") 
            event_hashes_dict[(event_hash, event_name)] = event_hashes_dict.get((event_hash, event_name), 0) + 1
    print("Unique events:")
    for event_info, count in event_hashes_dict.items():
        event_hash, event_name = event_info
        print(f"{event_name} ({event_hash}): {count}")

if __name__ == "__main__": 
    main()

Unique events:
Unknown (0xbd5318adc778260bd213cc16f3ef06cfb4f729adb9309495fed0e10abc61c375): 2
Unknown (0xcb34d0577abe5b2f96b65ea1bb5de2209ba6309c6909438c6d50dd277ca3b580): 2
Unknown (0x0730610a5322c6584fb6f5bb760719e650c888cfd965a2beb2d598bcd425e394): 81
Unknown (0xb34ee265e3d4f5ec4e8b52d59b2a9be8fceca2f274ebc080d8fba797fea9391f): 1700
Unknown (0x9dcff9d94fbfdb4622d11edb383005f95e78efb446c72d92f8e615c6025c4703): 111
Unknown (0x5229a5dba83a54ae8cb5b51bdd6de9474cacbe9dd332f5185f3a4f4f2e3f4ad9): 62
Unknown (0xbf8e2b108bb7c980e08903a8a46527699d5e84905a082d56dacb4150725c8cab): 62
Unknown (0x4d72fe0577a3a3f7da968d7b892779dde102519c25527b29cf7054f245c791b9): 3
